In [1]:
!pip install -U optuna-integration[tfkeras]
!pip install -U pytorch

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pytorch
Failed to build pytorch


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\79112\AppData\Local\Temp\pip-install-lpx99y62\pytorch_af256cc7e154476c82043e098369c0e2\setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects


In [1]:
import pandas as pd
import numpy as np
import warnings

from feature_engineering import AAVE, ARB, BNB, BTC, ETH, JASMY, LDO, LINK, OP, PENDLE
from data_preprocessing import prepare_data, prepare_for_train
from vizualize import show_example


import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate, Lambda, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler

In [81]:
warnings.simplefilter('ignore', Warning)

In [2]:
df = prepare_data(AAVE, multiplier=2).copy()
df

,timestamp,open,high,low,close,SMA_delta,RSI_14,BB_delta_cu,BB_delta_cl,ATR,Revers,Moment,next_high,next_low,next_close,dynamic_range,target
199,2023-01-02 22:45:00,53.50,53.50,53.20,53.20,-1.20850,30.769231,0.705370,-0.164630,0.224202,-0.000000,-0.003731,53.50,53.20,53.30,0.004214,1
200,2023-01-02 23:00:00,53.30,53.50,53.20,53.30,-1.19000,35.714286,0.627246,-0.012754,0.234309,-0.001878,-0.005623,53.40,53.30,53.30,0.004396,0
201,2023-01-02 23:15:00,53.30,53.40,53.30,53.30,-1.17150,35.714286,0.642491,0.032491,0.229734,-0.000000,0.000000,53.40,53.20,53.40,0.004310,0
202,2023-01-02 23:30:00,53.30,53.40,53.20,53.40,-1.15700,42.857143,0.544142,0.154142,0.212436,-0.000000,0.000000,53.40,52.90,53.00,0.003978,-1
203,2023-01-02 23:45:00,53.30,53.40,52.90,53.00,-1.12900,29.411765,1.004051,-0.135949,0.210778,0.001874,0.001874,53.10,52.90,53.00,0.003977,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34945,2023-12-30 22:30:00,112.08,112.52,111.85,112.41,1.04415,49.453552,0.521033,0.893033,0.726014,-0.000089,-0.001516,112.48,111.33,112.09,0.006459,-1
34946,2023-12-30 22:45:00,112.43,112.48,111.33,112.09,1.00110,44.783715,0.830014,0.531014,0.713212,-0.000178,0.002940,112.95,112.02,112.45,0.006363,1
34947,2023-12-30 23:00:00,112.08,112.95,112.02,112.45,0.92620,52.736318,0.450825,0.786825,0.834117,0.000089,-0.003029,112.51,112.03,112.15,0.007418,0
34948,2023-12-30 23:15:00,112.40,112.51,112.03,112.15,0.85815,53.132832,0.676968,0.348968,0.788235,0.000445,0.003296,112.33,111.49,111.50,0.007028,0


In [3]:
show_example(df)

In [4]:
prepare_for_train(df).reset_index(drop=True)

,open,high,low,close,SMA_delta,RSI_14,BB_delta_cu,BB_delta_cl,ATR,Revers,Moment,target
0,53.50,53.50,53.20,53.20,-1.20850,30.769231,0.705370,-0.164630,0.224202,-0.000000,-0.003731,1
1,53.30,53.50,53.20,53.30,-1.19000,35.714286,0.627246,-0.012754,0.234309,-0.001878,-0.005623,0
2,53.30,53.40,53.30,53.30,-1.17150,35.714286,0.642491,0.032491,0.229734,-0.000000,0.000000,0
3,53.30,53.40,53.20,53.40,-1.15700,42.857143,0.544142,0.154142,0.212436,-0.000000,0.000000,-1
4,53.30,53.40,52.90,53.00,-1.12900,29.411765,1.004051,-0.135949,0.210778,0.001874,0.001874,0
...,...,...,...,...,...,...,...,...,...,...,...,...
34746,112.08,112.52,111.85,112.41,1.04415,49.453552,0.521033,0.893033,0.726014,-0.000089,-0.001516,-1
34747,112.43,112.48,111.33,112.09,1.00110,44.783715,0.830014,0.531014,0.713212,-0.000178,0.002940,1
34748,112.08,112.95,112.02,112.45,0.92620,52.736318,0.450825,0.786825,0.834117,0.000089,-0.003029,0
34749,112.40,112.51,112.03,112.15,0.85815,53.132832,0.676968,0.348968,0.788235,0.000445,0.003296,0


In [5]:
df.drop('timestamp', axis=1, inplace=True)

In [6]:
df['target'] = df['target'].replace({-1: 1, 1: 2}) 

In [7]:
df = prepare_for_train(df)
df

,open,high,low,close,SMA_delta,RSI_14,BB_delta_cu,BB_delta_cl,ATR,Revers,Moment,target
199,53.50,53.50,53.20,53.20,-1.20850,30.769231,0.705370,-0.164630,0.224202,-0.000000,-0.003731,2
200,53.30,53.50,53.20,53.30,-1.19000,35.714286,0.627246,-0.012754,0.234309,-0.001878,-0.005623,0
201,53.30,53.40,53.30,53.30,-1.17150,35.714286,0.642491,0.032491,0.229734,-0.000000,0.000000,0
202,53.30,53.40,53.20,53.40,-1.15700,42.857143,0.544142,0.154142,0.212436,-0.000000,0.000000,1
203,53.30,53.40,52.90,53.00,-1.12900,29.411765,1.004051,-0.135949,0.210778,0.001874,0.001874,0
...,...,...,...,...,...,...,...,...,...,...,...,...
34945,112.08,112.52,111.85,112.41,1.04415,49.453552,0.521033,0.893033,0.726014,-0.000089,-0.001516,1
34946,112.43,112.48,111.33,112.09,1.00110,44.783715,0.830014,0.531014,0.713212,-0.000178,0.002940,2
34947,112.08,112.95,112.02,112.45,0.92620,52.736318,0.450825,0.786825,0.834117,0.000089,-0.003029,0
34948,112.40,112.51,112.03,112.15,0.85815,53.132832,0.676968,0.348968,0.788235,0.000445,0.003296,0


In [10]:
raise

RuntimeError: No active exception to reraise

In [53]:
import torch
import torch.nn as nn

class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, class_weights):
        super(WeightedCrossEntropyLoss, self).__init__()
        # Преобразуем class_weights в тензор и сохраняем
        self.class_weights = torch.tensor(class_weights, dtype=torch.float32)

    def forward(self, outputs, targets):
        targets = targets.long()  # Преобразуем в long для совместимости с CrossEntropyLoss
        # Переносим class_weights на устройство, соответствующее targets
        weights = self.class_weights.to(targets.device)
        # Используем глобальные веса для CrossEntropyLoss
        loss = nn.CrossEntropyLoss(weight=weights)(outputs, targets)
        return loss


In [54]:
class_weights = (df.target.value_counts() / df.shape[0]).values
class_weights

(0.1273344651952462, 0.7554027222238209, 0.11726281258093292)

In [62]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

class FinancialDataset(Dataset):
    def __init__(self, data, sequence_length, target_col):
        """
        data: pandas DataFrame с временными рядами и признаками
        sequence_length: длина временной последовательности
        target_col: название столбца, который является целевой переменной
        """
        self.data = data
        self.sequence_length = sequence_length
        self.target_col = target_col

        # Преобразуем данные в тензоры
        self.features = torch.tensor(data.drop(columns=[target_col]).values, dtype=torch.float32)
        self.targets = torch.tensor(data[target_col].values, dtype=torch.float32)

    def __len__(self):
        # Количество доступных последовательностей
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        # Последовательность признаков
        x = self.features[idx:idx + self.sequence_length]
        # Целевая переменная для предсказания
        y = self.targets[idx + self.sequence_length]
        # Добавляем размерность для y
        y = torch.tensor([y], dtype=torch.float32)  # Теперь y имеет форму [1]
        return x, y

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Прогоняем данные через LSTM
        lstm_out, _ = self.lstm(x)
        # Берем только выход последнего временного шага
        lstm_last_out = lstm_out[:, -1, :]
        # Полносвязный слой
        output = self.fc(lstm_last_out)
        return output


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model = model.to(device)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.squeeze())  # Убираем лишнюю размерность в targets
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Валидация
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets.squeeze())
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    return model

In [63]:
class TimeSeriesSplit:
    def __init__(self, test_size=0.2):
        self.test_size = test_size
        
    def split(self, data):
        split_idx = int(len(data)*(1-self.test_size))
        return data[:split_idx], data[split_idx:]


targets = df.target.values
features = df.drop(['target', 'open', 'high', 'low'], axis=1).values 

# Разделение на тренировочные и валидационные данные
x_train, x_val, y_train, y_val = train_test_split(features, targets, test_size=0.2, shuffle=False)

# Преобразование в тензоры
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_val = torch.tensor(x_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

# Создание Dataset
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

# Создание DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [65]:
sequence_length = 10
target_col = "target"  # Название столбца с метками
train_dataset = FinancialDataset(data=df, sequence_length=sequence_length, target_col=target_col)

x, y = train_dataset[0]
print(f"x shape: {x.shape}, y shape: {y.shape}")

x shape: torch.Size([10, 13]), y shape: torch.Size([1])


In [66]:
print(df[['open', 'high', 'low', 'close']].corr())

           open      high       low     close
open   1.000000  0.999793  0.999785  0.999684
high   0.999793  1.000000  0.999637  0.999812
low    0.999785  0.999637  1.000000  0.999830
close  0.999684  0.999812  0.999830  1.000000


In [59]:
print(f"Input shape: {x.shape}")

Input shape: torch.Size([10, 13])


In [68]:
# Параметры модели
input_size = 10  # Количество признаков
hidden_size = 64
num_layers = 2
num_classes = 3
class_weights = (df.target.value_counts() / df.shape[0]).values

# Инициализация модели
model = LSTMModel(input_size, hidden_size, num_layers, num_classes)

# Функция потерь и оптимизатор
criterion = WeightedCrossEntropyLoss(class_weights=class_weights)
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=1e-3,
    weight_decay=1e-3,
    amsgrad=True
)
scheduler = torch.optim.lr_scheduler.CyclicLR(
    optimizer, 
    base_lr=1e-5,
    max_lr=1e-3,
    step_size_up=500,
    cycle_momentum=False
)

# Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Обучение
num_epochs = 20
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)


Inputs shape: torch.Size([16, 10])
Targets shape: torch.Size([16])


IndexError: too many indices for tensor of dimension 2

In [75]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score # <---  Добавлен импорт
import numpy as np

# === Класс для взвешенной функции потерь ===
class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, class_weights):
        super(WeightedCrossEntropyLoss, self).__init__()
        self.class_weights = torch.tensor(class_weights, dtype=torch.float32)

    def forward(self, outputs, targets):
        targets = targets.long()  # Преобразуем в long
        weights = self.class_weights.to(targets.device)
        loss = nn.CrossEntropyLoss(weight=weights)(outputs, targets)
        return loss

# === Класс для LSTM ===
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_last_out = lstm_out[:, -1, :]  # Последний временной шаг
        output = self.fc(lstm_last_out)
        return output

# === Функция обучения ===
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model = model.to(device)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device).long()  # Приведение к long
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        val_loss = 0.0
        all_val_targets = [] # Добавлено для сбора целей валидации
        all_val_outputs = [] # Добавлено для сбора предсказаний валидации
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device).long()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                all_val_targets.extend(targets.cpu().numpy()) # Сбор целей
                all_val_outputs.extend(torch.argmax(outputs, dim=1).cpu().numpy()) # Сбор предсказаний

        avg_val_loss = val_loss / len(val_loader)

        # Расчет метрик на валидационном наборе
        val_accuracy = accuracy_score(all_val_targets, all_val_outputs)
        val_f1_macro = f1_score(all_val_targets, all_val_outputs, average='macro', zero_division=0) # Важно zero_division=0

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1 Macro: {val_f1_macro:.4f}") # Вывод метрик

    return model

# === Подготовка данных ===
# Генерация фиктивных данных (замените на ваш DataFrame df)
num_samples = 1000
num_features = 10
num_classes = 3

np.random.seed(42)
features = df.drop(['target', 'open', 'high', 'low'], axis=1).values 
targets = df.target.values

# Разделение на train и val
x_train, x_val, y_train, y_val = train_test_split(features, targets, test_size=0.2, shuffle=False)

# Преобразование в тензоры
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
x_val = torch.tensor(x_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

# Создание Dataset и DataLoader
train_dataset = TensorDataset(x_train.unsqueeze(1), y_train)  # Добавляем временную размерность
val_dataset = TensorDataset(x_val.unsqueeze(1), y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# === Инициализация модели ===
input_size = x_train.shape[1]  # Количество признаков
hidden_size = 64
num_layers = 2
num_classes = 3
class_weights = np.bincount(y_train.numpy()) / len(y_train)

model = LSTMModel(input_size, hidden_size, num_layers, num_classes)
criterion = WeightedCrossEntropyLoss(class_weights)
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-3,
    amsgrad=True,
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Обучение модели ===
num_epochs = 20
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)


Epoch 1/20, Train Loss: 0.2595, Val Loss: 0.2751, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 2/20, Train Loss: 0.2490, Val Loss: 0.2746, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 3/20, Train Loss: 0.2470, Val Loss: 0.2742, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 4/20, Train Loss: 0.2455, Val Loss: 0.2725, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 5/20, Train Loss: 0.2446, Val Loss: 0.2712, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 6/20, Train Loss: 0.2439, Val Loss: 0.2697, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 7/20, Train Loss: 0.2434, Val Loss: 0.2685, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 8/20, Train Loss: 0.2430, Val Loss: 0.2683, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 9/20, Train Loss: 0.2428, Val Loss: 0.2675, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 10/20, Train Loss: 0.2424, Val Loss: 0.2672, Val Accuracy: 0.7487, Val F1 Macro: 0.2854
Epoch 11/20, Train Loss: 0.2421, Val Loss: 0.2664, Val Accuracy: 0.74

In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, class_weights):
        super(WeightedCrossEntropyLoss, self).__init__()
        self.class_weights = torch.tensor(class_weights, dtype=torch.float32)

    def forward(self, outputs, targets):
        targets = targets.long()
        weights = self.class_weights.to(targets.device)
        loss = nn.CrossEntropyLoss(weight=weights)(outputs, targets)
        return loss

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.batch_norm = nn.BatchNorm1d(hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size // 2, num_classes)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        lstm_out, _ = self.lstm(x, (h0, c0))
        
        # Get last time step output
        last_hidden = lstm_out[:, -1, :]
        
        # Apply batch normalization
        normalized = self.batch_norm(last_hidden)
        
        # First dense layer
        out = self.fc1(normalized)
        out = self.relu(out)
        out = self.dropout(out)
        
        # Output layer
        out = self.fc2(out)
        return out

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, patience=5):
    model = model.to(device)
    best_val_f1 = 0
    patience_counter = 0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_predictions = []
        train_targets = []

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.squeeze())
            
            loss.backward()
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
            train_predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_targets.extend(targets.cpu().numpy())

        avg_train_loss = train_loss / len(train_loader)
        train_metrics = calculate_metrics(np.array(train_targets), np.array(train_predictions))

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_predictions = []
        val_targets = []

        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets.squeeze())
                
                val_loss += loss.item()
                val_predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_targets.extend(targets.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        val_metrics = calculate_metrics(np.array(val_targets), np.array(val_predictions))

        # Early stopping check
        if val_metrics['f1_macro'] > best_val_f1:
            best_val_f1 = val_metrics['f1_macro']
            patience_counter = 0
        else:
            patience_counter += 1

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train - Loss: {avg_train_loss:.4f}, Accuracy: {train_metrics['accuracy']:.4f}, F1: {train_metrics['f1_macro']:.4f}")
        print(f"Val - Loss: {avg_val_loss:.4f}, Accuracy: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1_macro']:.4f}")
        
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

    return model

def calculate_metrics(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1_macro': f1_score(y_true, y_pred, average='macro', labels=[0, 1, 2], zero_division=0),
        'precision': precision_score(y_true, y_pred, average=None, labels=[0, 1, 2], zero_division=0),
        'recall': recall_score(y_true, y_pred, average=None, labels=[0, 1, 2], zero_division=0)
    }


# Гиперпараметры
input_size = 10  # Размерность входных данных
hidden_size = 128
num_layers = 2
num_classes = 3
learning_rate = 0.0001
num_epochs = 100
batch_size = 32

# Веса классов для несбалансированных данных
class_weights = [1.0, 1.0, 1.0]  # Настройте в соответствии с распределением классов

# Инициализация модели и оптимизатора
model = LSTMModel(input_size, hidden_size, num_layers, num_classes)
criterion = WeightedCrossEntropyLoss(class_weights)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)


Epoch 1/100
Train - Loss: 0.7404, Accuracy: 0.7556, F1: 0.2902
Val - Loss: 0.8516, Accuracy: 0.7487, F1: 0.2854
Epoch 2/100
Train - Loss: 0.7207, Accuracy: 0.7572, F1: 0.2889
Val - Loss: 0.8796, Accuracy: 0.7006, F1: 0.3359
Epoch 3/100
Train - Loss: 0.7106, Accuracy: 0.7568, F1: 0.2918
Val - Loss: 0.9245, Accuracy: 0.7054, F1: 0.3350
Epoch 4/100
Train - Loss: 0.7027, Accuracy: 0.7566, F1: 0.2965
Val - Loss: 0.9762, Accuracy: 0.7282, F1: 0.3379
Epoch 5/100
Train - Loss: 0.6983, Accuracy: 0.7572, F1: 0.3018
Val - Loss: 1.0367, Accuracy: 0.7261, F1: 0.3358
Epoch 6/100
Train - Loss: 0.6936, Accuracy: 0.7572, F1: 0.3057
Val - Loss: 0.9514, Accuracy: 0.7395, F1: 0.3303
Epoch 7/100
Train - Loss: 0.6873, Accuracy: 0.7584, F1: 0.3212
Val - Loss: 1.1080, Accuracy: 0.7218, F1: 0.3429
Epoch 8/100
Train - Loss: 0.6860, Accuracy: 0.7581, F1: 0.3266
Val - Loss: 1.1035, Accuracy: 0.7300, F1: 0.3405
Epoch 9/100
Train - Loss: 0.6809, Accuracy: 0.7592, F1: 0.3383
Val - Loss: 1.1337, Accuracy: 0.7196, F1:

KeyboardInterrupt: 

In [82]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

# --- 1. Усиление взвешивания классов (Weighted Loss - already implemented, showing how to adjust weights) ---
# В вашем коде WeightedCrossEntropyLoss уже реализована поддержка весов классов.
# Чтобы усилить взвешивание классов меньшинства, нужно изменить значения `class_weights`.

# Пример изменения весов классов (увеличьте веса для классов 1 и 2):
class_weights = np.array([1.0, 3.0, 3.0]) # Пример: класс 0 - вес 1, классы 1 и 2 - вес 3
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device) # device should be defined already

criterion = WeightedCrossEntropyLoss(class_weights=class_weights_tensor) # Используйте обновленные веса при создании criterion


# --- 2. Bidirectional LSTM (Обновленный класс LSTMModel) ---
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.0): # Added dropout
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=dropout) # Bidirectional and dropout
        self.fc = nn.Linear(hidden_size * 2, num_classes) # x2 because bidirectional

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_last_out = lstm_out[:, -1, :]
        output = self.fc(lstm_last_out)
        return output


# --- 3. Optuna с оптимизацией window_size (Обновленная objective функция) ---
def objective(trial):
    # Гиперпараметры для оптимизации
    hidden_size = trial.suggest_int("hidden_size", 64, 256, step=64)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    window_size = trial.suggest_int("window_size", 10, 60, step=10) # <--- Добавлено window_size в пространство поиска
    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.1) # Dropout rate

    # Подготовка данных с динамическим window_size
    X, y = create_sequences(scaled_data, window_size) # Используйте trial.suggest_int("window_size", ...)
    split = int(0.8 * len(X))
    x_train_trial, x_val_trial = torch.tensor(X[:split], dtype=torch.float32), torch.tensor(X[split:], dtype=torch.float32)
    y_train_trial, y_val_trial = torch.tensor(y[:split], dtype=torch.float32), torch.tensor(y[split:], dtype=torch.float32)

    # Dataset и DataLoader с динамическим window_size
    train_dataset_trial = TensorDataset(x_train_trial, y_train_trial)
    val_dataset_trial = TensorDataset(x_val_trial, y_val_trial)
    train_loader_trial = DataLoader(train_dataset_trial, batch_size=batch_size, shuffle=True)
    val_loader_trial = DataLoader(val_dataset_trial, batch_size=batch_size, shuffle=False)


    # Модель, Оптимизатор, Критерий
    model = LSTMModel(input_size=x_train_trial.shape[2], hidden_size=hidden_size, num_layers=num_layers, num_classes=3, dropout=dropout) # Pass dropout
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-3, amsgrad=True)
    criterion = WeightedCrossEntropyLoss(class_weights=class_weights_tensor)

    # Обучение модели
    num_epochs = 5 # Уменьшено для Optuna
    trained_model = train_model(model, train_loader_trial, val_loader_trial, criterion, optimizer, num_epochs, device)

    # Валидация и расчет метрики
    model.eval()
    all_val_preds = []
    all_val_targets = []
    with torch.no_grad():
        for inputs, targets in val_loader_trial: # Use val_loader_trial
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = trained_model(inputs)
            all_val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            all_val_targets.extend(targets.cpu().numpy())

    accuracy = accuracy_score(all_val_targets, all_val_preds)
    return accuracy


# ---- Запуск Optuna ----
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Assuming df, class_weights, train_dataset, val_dataset, x_train, scaled_data are already defined
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10, timeout=3600) # Reduced trials for example

    print("Best trial:")
    print(study.best_trial.params)
    print("Best Accuracy:", study.best_value)

    best_params = study.best_trial.params


C:\Users\79112\AppData\Local\Temp\ipykernel_20068\1837967402.py:11: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

[I 2025-01-26 14:35:57,715] A new study created in memory with name: no-name-0a1c9873-2233-4a57-a174-0a93f423182e
C:\Users\79112\AppData\Local\Temp\ipykernel_20068\1310814379.py:43: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[W 2025-01-26 14:35:57,718] Trial 0 failed with parameters: {'hidden_size': 192, 'num_layers': 3, 'learning_rate': 0.0039789128297440515, 'batch_size': 64, 'window_size': 10, 'dropout': 0.5} because of the following error: NameError("name 'create_sequences' is not defined").
Traceback (most recent call last):
  File "c:\Users\79112\.conda\envs

NameError: name 'create_sequences' is not defined

In [91]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import os

class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, class_weights):
        super(WeightedCrossEntropyLoss, self).__init__()
        self.class_weights = torch.tensor(class_weights, dtype=torch.float32)

    def forward(self, outputs, targets):
        targets = targets.long()
        weights = self.class_weights.to(targets.device)
        loss = nn.CrossEntropyLoss(weight=weights)(outputs, targets)
        return loss

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Инициализация скрытого и клеточного состояний
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Прямой проход через LSTM
        lstm_out, _ = self.lstm(x, (h0, c0))
        
        # Получение последнего временного шага
        last_hidden = lstm_out[:, -1, :]
        
        # Пропуск через Dropout
        out = self.dropout(last_hidden)
        
        # Выходной слой
        out = self.fc(out)
        return out

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, patience=10):
    model = model.to(device)
    best_val_f1 = 0
    patience_counter = 0
    
    for epoch in range(num_epochs):
        # Фаза обучения
        model.train()
        train_loss = 0.0
        train_predictions = []
        train_targets = []

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.squeeze())
            
            loss.backward()
            # Ограничение градиентов для предотвращения взрыва градиентов
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
            train_predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_targets.extend(targets.cpu().numpy())

        avg_train_loss = train_loss / len(train_loader)
        train_metrics = calculate_metrics(np.array(train_targets), np.array(train_predictions))

        # Фаза валидации
        model.eval()
        val_loss = 0.0
        val_predictions = []
        val_targets = []

        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets.squeeze())
                
                val_loss += loss.item()
                val_predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_targets.extend(targets.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        val_metrics = calculate_metrics(np.array(val_targets), np.array(val_predictions))

        # Раннее прекращение обучения
        if val_metrics['f1_macro'] > best_val_f1:
            best_val_f1 = val_metrics['f1_macro']
            patience_counter = 0
            # Сохранение лучшей модели
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train - Loss: {avg_train_loss:.4f}, Accuracy: {train_metrics['accuracy']:.4f}, F1: {train_metrics['f1_macro']:.4f}")
        print(f"Val   - Loss: {avg_val_loss:.4f}, Accuracy: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1_macro']:.4f}")
        
        if patience_counter >= patience:
            print("Ранняя остановка активирована")
            break

    # Загрузка лучшей модели
    model.load_state_dict(torch.load('best_model.pth'))
    return model

def calculate_metrics(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1_macro': f1_score(y_true, y_pred, average='macro', labels=[0, 1, 2], zero_division=0),
        'precision': precision_score(y_true, y_pred, average=None, labels=[0, 1, 2], zero_division=0),
        'recall': recall_score(y_true, y_pred, average=None, labels=[0, 1, 2], zero_division=0)
    }

def prepare_data(file_paths):
    import pandas as pd
    from sklearn.preprocessing import StandardScaler, LabelEncoder

    features = df.drop(['target', 'open', 'high', 'low'], axis=1).values 
    targets = df.target.values
    
    # Нормализация
    scaler = StandardScaler()
    features = scaler.fit_transform(features)
    
    all_data = [features, targets]
    
    # Объединение всех данных
    X = all_data[0]
    y = all_data[1]
    
    return X, y

def create_data_loaders(X, y, sequence_length=10, batch_size=32, split_ratio=0.8):
    # Создание последовательностей для LSTM
    sequences = []
    targets = []
    for i in range(len(X) - sequence_length):
        sequences.append(X[i:i+sequence_length])
        targets.append(y[i+sequence_length])
    
    sequences = np.array(sequences)
    targets = np.array(targets)
    
    # Разделение на обучающую и валидационную выборки
    split = int(len(sequences) * split_ratio)
    X_train, X_val = sequences[:split], sequences[split:]
    y_train, y_val = targets[:split], targets[split:]
    
    # Создание TensorDataset
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
    
    # Создание DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader

# Основной блок кода
if __name__ == "__main__":
    # Пути к файлам с данными
    file_paths = sv_files = [f for f in os.listdir(
        "./data/") if os.path.isfile(os.path.join("./data/", f))]
    
    # Подготовка данных
    X, y = prepare_data(file_paths)
    
    # Создание DataLoader
    train_loader, val_loader = create_data_loaders(X, y, sequence_length=10, batch_size=32, split_ratio=0.8)
    
    # Гиперпараметры
    input_size = X.shape[1]  # Количество признаков
    hidden_size = 64
    num_layers = 2
    num_classes = 3
    learning_rate = 0.0005
    num_epochs = 100
    batch_size = 32
    
    # Рассчёт весов классов для обработки дисбаланса
    class_counts = np.bincount(y)
    class_weights = 1. / (class_counts + 1e-6)  # Добавление малой величины для избежания деления на ноль
    class_weights = class_weights / class_weights.sum() * len(class_weights)  # Нормализация
    
    # Инициализация модели и оптимизатора
    model = LSTMModel(input_size, hidden_size, num_layers, num_classes, dropout=0.3)
    criterion = WeightedCrossEntropyLoss(class_weights)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Обучение модели
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, patience=10)
    
    # Сохранение финальной модели
    torch.save(model.state_dict(), 'final_model.pth')


Epoch 1/100
Train - Loss: 1.0848, Accuracy: 0.5025, F1: 0.3472
Val   - Loss: 1.0800, Accuracy: 0.5053, F1: 0.3497
Epoch 2/100
Train - Loss: 1.0764, Accuracy: 0.5263, F1: 0.3655
Val   - Loss: 1.0782, Accuracy: 0.5214, F1: 0.3145
Epoch 3/100
Train - Loss: 1.0722, Accuracy: 0.5464, F1: 0.3698
Val   - Loss: 1.0784, Accuracy: 0.5091, F1: 0.3357
Epoch 4/100
Train - Loss: 1.0704, Accuracy: 0.5369, F1: 0.3731
Val   - Loss: 1.0742, Accuracy: 0.5018, F1: 0.3351
Epoch 5/100
Train - Loss: 1.0697, Accuracy: 0.5379, F1: 0.3760
Val   - Loss: 1.0769, Accuracy: 0.4465, F1: 0.3201
Epoch 6/100
Train - Loss: 1.0670, Accuracy: 0.5405, F1: 0.3798
Val   - Loss: 1.0781, Accuracy: 0.5424, F1: 0.3383
Epoch 7/100
Train - Loss: 1.0663, Accuracy: 0.5489, F1: 0.3858
Val   - Loss: 1.0883, Accuracy: 0.4064, F1: 0.2876
Epoch 8/100
Train - Loss: 1.0656, Accuracy: 0.5456, F1: 0.3837
Val   - Loss: 1.0783, Accuracy: 0.4929, F1: 0.3366
Epoch 9/100
Train - Loss: 1.0647, Accuracy: 0.5463, F1: 0.3798
Val   - Loss: 1.0755, Acc

C:\Users\79112\AppData\Local\Temp\ipykernel_20068\3906777930.py:124: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader



# Предположим, что входные признаки в колонках 'feature1', 'feature2', ..., а цель в колонках 'target_0', 'target_1', 'target_2'
features = df.iloc[:, 3:].values
targets = df.iloc[:, :3].values  # One-hot encoded target

# Разделение на тренировочные и валидационные данные
x_train, x_val, y_train, y_val = train_test_split(features, targets, test_size=0.2, random_state=42)

# Преобразование в тензоры
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
x_val = torch.tensor(x_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)


In [ ]:
print("Shape of x:", features.shape)  # Ожидается (N, 11)
print("Shape of y:", targets.shape)  # Ожидается (N, 3)

Shape of x: (34751, 11)
Shape of y: (34751, 3)


In [ ]:
# Создание Dataset
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

# Создание DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch.nn as nn
import torch.optim as optim

class PricePredictionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(PricePredictionLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)  # Выходное число классов

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # Последний выход LSTM
        out = self.fc(out)
        return out


seq_length = 1  # Если данные не являются последовательностями, можно использовать seq_length = 1
x_train = x_train.view(x_train.shape[0], seq_length, x_train.shape[1])
x_val = x_val.view(x_val.shape[0], seq_length, x_val.shape[1])

# Параметры модели
input_dim = x_train.shape[2]  # Количество признаков (feature_size)
hidden_dim = 64
output_dim = 3  # Количество классов
num_layers = 2
dropout = 0.2
num_epochs = 50
learning_rate = 0.001

# Создаем модель
model = PricePredictionLSTM(input_size=input_dim, hidden_size=hidden_dim, num_classes=output_dim)
loss_fn = nn.CrossEntropyLoss()

# Пример вычисления ошибки
y_train_classes = torch.argmax(y_train, dim=1)  # Преобразование one-hot в индексы
outputs = model(x_train)
loss = loss_fn(outputs, y_train_classes)

# Оптимизатор
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Обучение модели
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        # Добавляем измерение для sequence_length
        x_batch = x_batch.unsqueeze(1)

        # Преобразование меток
        y_batch_classes = torch.argmax(y_batch, dim=1)

        # Прямой проход
        outputs = model(x_batch)
        loss = loss_fn(outputs, y_batch_classes)

        # Обратное распространение
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Валидация
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x_val_batch, y_val_batch in val_loader:
            x_val_batch, y_val_batch = x_val_batch.to(device), y_val_batch.to(device)

            # Добавляем измерение для sequence_length
            x_val_batch = x_val_batch.unsqueeze(1)

            y_val_classes = torch.argmax(y_val_batch, dim=1)
            outputs = model(x_val_batch)
            val_loss += loss_fn(outputs, y_val_classes).item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss / len(val_loader):.4f}")

RuntimeError: shape '[27800, 1, 1]' is invalid for input of size 305800

In [ ]:
print(x_train.shape)


torch.Size([27800, 1, 11])


In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

# Определение целевой функции
def objective(trial):
    # Гиперпараметры, которые будет выбирать Optuna
    input_size = 11  # Количество фичей (OHLC + индикаторы)
    hidden_size = trial.suggest_int("hidden_size", 32, 256, step=32)  # Скрытые нейроны
    num_layers = trial.suggest_int("num_layers", 1, 3)  # Количество слоев
    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.1)  # Dropout
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)  # LR
    batch_size = trial.suggest_int("batch_size", 16, 128, step=16)  # Размер батча

    # Создаем модель
    model = PricePredictionLSTM(input_size, hidden_size, num_layers, output_size=3, dropout=dropout)
    model.to(device)

    # Оптимизатор
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Загрузка данных
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Обучение модели
    for epoch in range(10):  # Число эпох (можно увеличить)
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Валидация
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_val, y_val in val_loader:
            X_val, y_val = X_val.to(device), y_val.to(device)
            preds = model(X_val).argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_val.cpu().numpy())

    # Возвращаем метрику для минимизации (например, -accuracy для максимизации)
    accuracy = accuracy_score(all_labels, all_preds)
    return -accuracy

In [ ]:
study = optuna.create_study(direction="maximize")  # Максимизация accuracy
study.optimize(objective, n_trials=50)  # Количество попыток

# Лучшая комбинация гиперпараметров
print("Best trial:")
print(study.best_trial.params)

In [ ]:
best_params = study.best_trial.params

# Создаем модель с лучшими параметрами
final_model = PricePredictionLSTM(
    input_size=11,
    hidden_size=best_params["hidden_size"],
    num_layers=best_params["num_layers"],
    output_size=3,
    dropout=best_params["dropout"],
)
final_model.to(device)

# Обучаем модель заново на всех данных с лучшими параметрами
train_loader = DataLoader(full_train_dataset, batch_size=best_params["batch_size"], shuffle=True)
optimizer = optim.Adam(final_model.parameters(), lr=best_params["learning_rate"])
# ... (Обучение модели как ранее)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

# 1. Custom Loss and Metrics
class BalancedCategoricalLoss(tf.keras.losses.Loss):
    def __init__(self, class_weights, name="balanced_categorical_loss"):
        super().__init__(name=name)
        self.class_weights = tf.constant(class_weights, dtype=tf.float32)

    def call(self, y_true, y_pred):
        weights = tf.reduce_sum(y_true * self.class_weights, axis=1)
        ce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        return tf.reduce_mean(ce * weights)

class F1Metrics(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val.argmax(axis=1)
        self.classes = np.unique(np.concatenate([self.y_val, [0, 1, 2]]))  # Гарантируем все классы

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_val).argmax(axis=1)
        
        # Добавляем защиту от отсутствующих классов
        present_classes = np.unique(np.concatenate([y_pred, self.y_val]))
        f1 = f1_score(self.y_val, y_pred, average=None, labels=[0, 1, 2], zero_division=0)
        
        logs['val_f1_0'] = f1[0]
        logs['val_f1_1'] = f1[1] if len(f1) > 1 else 0
        logs['val_f1_2'] = f1[2] if len(f1) > 2 else 0
        
        print(f"\nVal F1: [0: {f1[0]:.3f}, -1: {f1[1]:.3f} (1), +1: {f1[2]:.3f} (2)] | Classes present: {present_classes}")

# 2. Data Preparation
def create_sequences(data, window_size=10):
    X, y = [], []
    for i in range(len(data)-window_size):
        X.append(data.iloc[i:i+window_size, 3:].values)
        y.append(data.iloc[i+window_size, :3].values)
    return np.array(X), np.array(y)


window_size = 15
X, y = create_sequences(df, window_size)

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size=0.2,
    shuffle=False  # Для временных рядов
)
# 3. Class Weight Calculation
class_counts = np.array([26251, 4425, 4075])
class_weights = (1 / class_counts) * (class_counts.sum() / 3)
class_weights /= class_weights.sum()  # Нормализация

# 4. Model Architecture
model = Sequential([
    LSTM(32, input_shape=(window_size, X.shape[2]), 
         kernel_regularizer=l2(0.005), recurrent_regularizer=l2(0.005)),
    Dropout(0.4),
    Dense(3, activation='softmax')
])

# 5. Model Compilation
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(
    optimizer=optimizer,
    loss=BalancedCategoricalLoss(class_weights),
    metrics=['accuracy']
)

# 6. Training Setup
callbacks = [
    EarlyStopping(monitor='val_f1_1', patience=10, mode='max', restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3),
    F1Metrics(X_val=X_test, y_val=y_test)
]

# 7. Training Execution
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=256,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    shuffle=False
)

# 8. Post-Training Analysis
def plot_training_history(history):
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Loss Evolution')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['val_f1_0'], label='F1 Class 0')
    plt.plot(history.history['val_f1_1'], label='F1 Class -1')
    plt.plot(history.history['val_f1_2'], label='F1 Class +1')
    plt.title('F1 Scores')
    plt.legend()
    
    plt.show()

plot_training_history(history)

# 9. Prediction and Risk Management
def calculate_levels(entry_price, atr, min_range=0.0035, max_range=0.02):
    range_size = np.clip(atr, min_range, max_range)
    return {
        'stop_loss': entry_price * (1 - range_size),
        'take_profit': entry_price * (1 + range_size)
    }

# 10. Model Evaluation
def full_evaluation(model, X, y):
    y_pred = model.predict(X).argmax(axis=1)
    y_true = y.argmax(axis=1)
    
    print("Classification Report:")
    print(classification_report(y_true, y_pred))
    
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    
    # Визуализация распределения предсказаний
    plt.figure(figsize=(10, 5))
    plt.hist(y_pred, bins=[-0.5, 0.5, 1.5, 2.5], alpha=0.7)
    plt.xticks([0, 1, 2], ['0', '-1', '+1'])
    plt.title("Prediction Distribution")
    plt.show()

full_evaluation(model, X_test, y_test)

# 11. Adaptive Trading System
def trading_simulation(model, data, initial_balance=10000):
    balance = initial_balance
    position = None
    trade_log = []
    
    for i in range(len(data)-window_size):
        sequence = data.iloc[i:i+window_size, 3:].values.reshape(1, window_size, -1)
        prediction = model.predict(sequence).argmax()
        current_price = data.iloc[i+window_size]['Close']
        
        # Логика управления позицией
        if position is None:
            if prediction == 1:  # Long
                position = {
                    'type': 'long',
                    'entry': current_price,
                    'sl': current_price * 0.997,
                    'tp': current_price * 1.003
                }
            elif prediction == 2:  # Short
                position = {
                    'type': 'short',
                    'entry': current_price,
                    'sl': current_price * 1.003,
                    'tp': current_price * 0.997
                }
        else:
            # Проверка условий закрытия
            if (position['type'] == 'long' and 
               (current_price <= position['sl'] or current_price >= position['tp'])):
                pnl = (current_price - position['entry']) / position['entry']
                balance *= (1 + pnl)
                trade_log.append(pnl)
                position = None
                
            elif (position['type'] == 'short' and 
                 (current_price >= position['sl'] or current_price <= position['tp'])):
                pnl = (position['entry'] - current_price) / position['entry']
                balance *= (1 + pnl)
                trade_log.append(pnl)
                position = None
                
    return balance, trade_log

# Запуск симуляции
final_balance, trades = trading_simulation(model, df)
print(f"Final balance: ${final_balance:.2f}")
print(f"Total trades: {len(trades)}")
print(f"Win rate: {sum(1 for x in trades if x > 0)/len(trades)*100:.1f}%")

MemoryError: Unable to allocate 2.92 MiB for an array with shape (11, 34751) and data type float64

In [ ]:
def feature_importance_analysis(model, X_test, y_test):
    baseline_score = model.evaluate(X_test, y_test, verbose=0)[1]
    importance = np.zeros((X.shape[2],))
    
    for i in range(X.shape[2]):
        X_temp = X_test.copy()
        np.random.shuffle(X_temp[:, :, i])
        shuffled_score = model.evaluate(X_temp, y_test, verbose=0)[1]
        importance[i] = baseline_score - shuffled_score
    
    return importance

feature_importance = feature_importance_analysis(model, X_test, y_test)
selected_features = np.where(feature_importance > 0.005)[0]

In [ ]:
important_features = df.columns[3:][selected_features]  # Получаем названия фичей
filtered_df = df.iloc[:, list(range(3)) + selected_features.tolist()]  # Новый датафрейм

In [ ]:
Сейчас я тебе предоставлю контекст для решения задачи, а в дальнейшем ты будешь должен ответить на мои вопросы и в итоге совместно со мной написать решение задачи на языке Python.

Мне необходимо разработать алгоритм-индикатор, который прогнозирует краткосрочные ценовые изменения выбранного финансового инструмента. Основная цель — предсказать направление (рост, падение или стабильность) цены на 15-минутный временной интервал с максимально возможной точностью. Модель должна генерировать рекомендации по уровням stop-loss и take-profit для каждой сделки.

Входные данные: В качестве данных используется выгрузка с биржи о движении цен в формате CSV. Данные хранятся в 10 файлах, которые содержат Open High, Low, Close и соответствующий timestamp для 10 различных активов. В каждом файле данные о конкретном активе. Суммарный объем данных составляет около 16 Мб. Данные предоставляются в таймфрейме 15-минут (интервал между данными составляет 15 минут).

Требования к решению (в порядке убывания приоритета):

Предсказать направление цены (+1: рост, -1: падение, 0: стабильность на следующий временной интервал.

Рассчитать оптимальные уровни stop-loss и take-profit для минимизации риска и максимизации прибыли.

Сделки должны идти непрерывно: одна сделка закрылась - следующая открылась.

Решение будет оцениваться в соответствии со следующими метриками (в порядке убывания приоритета):

Win-rate – процент прибыльных сделок.

PnL (Profit and Loss) – итоговая прибыль/убыток в процентах за определенный период.

Количество сделок – количество всех сделок, совершенных в процессе торговли.

Максимальная просадка депозита – максимальное снижение капитала за весь период торговли.

Средний процент прибыли на сделку – средний процент, полученный за каждую прибыльную сделку.

Средняя ширина от точки входа до выхода – средний процент изменения цены от входа в сделку до ее завершения.

Дополнительные условия:

Ограничений на выбор алгоритмов нет, но крайне приветствуются интерпретируемые модели и модели машинного обучения.

Stop-loss и take-profit должны быть рассчитаны на основе исторических данных, волатильности, прогнозов модели и других параметров. А ширина от точки входа до stop-loss и take-profit ордеров должна быть фиксирована (risk/reward 1:1). Причем, минимальный диапазон 0.35%, максимальный 2%.

Таким образом, цель задания - разработать алгоритм, который анализирует данные о движении цены криптовалют и автоматически рассчитывает возможные уровни для входа в сделку, стоп-лосса и тейк-профита.

Необязательные, но важные предпочтения:

Минимальная задержка в обработке данных (реализация модели должна быть пригодна для реального времени).

Реализация адаптивного расчета stop-loss и take-profit с учетом изменения волатильности в реальном времени

Нормализация данных (избавление от шумов и аномальных движений).

В качестве дополнительных фичей, помимо High, Open, Close и Low,  я собираюсь использовать следующие индикаторы: SMA_delta (Дельта скользящих), RSI_14 (Индекс относительной силы), BB_delta_cu, BB_delta_cl, ATR, Mean-reversion, Momentum. Я выбрал стратегию прогнозирования High/Low, а не Close-to-Close. Я хочу обучить LSTM для этих целей. 

Твои задачи:
1. Придумать и разработаь функцию потерь, которая будет максимизировать Win Rate, то есть % верных предсказаний +1, -1 и 0 от общего количества предсказаний.
2. Напиши код для обучения LSTM
3. Напиши код, который будет оптимизировать гиперпараметры модели с помощью Optuna, Hyperopt или подобного инструмента.
4. Напиши код, который будет отбирать фичи

SyntaxError: invalid character '—' (U+2014) (3141982048.py, line 3)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, concatenate, BatchNormalization, LayerNormalization, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import RobustScaler
from sklearn.utils.class_weight import compute_class_weight
import optuna
from optuna.integration import TFKerasPruningCallback
import matplotlib.pyplot as plt

# 1. Кастомная метрика для точности направления
class DirectionAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='direction_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.correct = self.add_weight(name='correct', initializer='zeros')
        self.total = self.add_weight(name='total', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_class = tf.argmax(y_pred, axis=1)
        y_true_class = tf.argmax(y_true, axis=1)
        matches = tf.cast(tf.equal(y_pred_class, y_true_class), tf.float32)
        self.correct.assign_add(tf.reduce_sum(matches))
        self.total.assign_add(tf.cast(tf.size(matches), tf.float32))

    def result(self):
        return self.correct / self.total

    def reset_states(self):
        self.correct.assign(0.0)
        self.total.assign(0.0)

# 2. Улучшенная функция потерь
def asymmetric_focal_loss(y_true, y_pred, alpha=[1.0, 1.0, 1.0], gamma=2.0):
    ce_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    probs = tf.reduce_max(y_true * y_pred, axis=1)
    alpha_weights = tf.reduce_sum(y_true * alpha, axis=1)
    focal_loss = alpha_weights * (1 - probs)**gamma * ce_loss
    return tf.reduce_mean(focal_loss)

# 3. Генератор временных окон с аугментацией
def create_sequences(data, window_size, step=1):
    sequences = []
    targets_dir = []
    targets_high = []
    targets_low = []

    for i in range(window_size, len(data)-1, step):
        sequences.append(data[i-window_size:i])
        targets_dir.append(data[i, 0:3])
        targets_high.append(data[i+1, 3])  # Прогноз на следующий шаг
        targets_low.append(data[i+1, 4])

    return (np.array(sequences),
            [np.array(targets_dir),
             np.array(targets_high).reshape(-1, 1),
             np.array(targets_low).reshape(-1, 1)])

# 4. Улучшенная архитектура модели
def create_enhanced_model(input_shape, lstm_units1, lstm_units2, dropout_rate):
    inputs = Input(shape=input_shape)

    # Временная ветвь с Bidirectional LSTM
    x = Bidirectional(LSTM(lstm_units1, return_sequences=True, dropout=dropout_rate))(inputs)
    x = LayerNormalization()(x)
    x = Bidirectional(LSTM(lstm_units2, dropout=dropout_rate))(x)

    # Технические индикаторы
    tech = inputs[:, -1, -3:]
    tech = Dense(32, activation='swish')(tech)

    # Объединение
    merged = concatenate([x, tech])
    merged = Dense(128, activation='swish')(merged)
    merged = Dropout(dropout_rate)(merged)

    # Выходы
    direction = Dense(3, activation='softmax', name='direction')(merged)    
    high = Dense(1, activation='linear', name='high')(merged)
    low = Dense(1, activation='linear', name='low')(merged)

    return Model(inputs, [direction, high, low])

# 5. Оптимизация гиперпараметров
def objective(trial):
    # Гиперпараметры
    hp = {
        'window_size': trial.suggest_int('window_size', 30, 90),
        'lstm_units1': trial.suggest_categorical('lstm_units1', [64, 128, 256]),
        'lstm_units2': trial.suggest_categorical('lstm_units2', [32, 64, 128]),
        'dropout_rate': trial.suggest_float('dropout_rate', 0.2, 0.5),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True),
        'gamma': trial.suggest_float('gamma', 1.0, 3.0),
        'batch_size': trial.suggest_categorical('batch_size', [64, 128]),
        'loss_weights_dir': trial.suggest_float('loss_weights_dir', 0.4, 0.8)
    }

    # Подготовка данных
    X, y = create_sequences(scaled_data, hp['window_size'])
    split = int(0.8 * len(X))
    X_train, X_val = X[:split], X[split:]
    y_train, y_val = [y[0][:split], y[1][:split], y[2][:split]], \
                     [y[0][split:], y[1][split:], y[2][split:]]
    
    # Модель
    model = create_enhanced_model(
        input_shape=(hp['window_size'], data.shape[1]),
        lstm_units1=hp['lstm_units1'],
        lstm_units2=hp['lstm_units2'],
        dropout_rate=hp['dropout_rate']
    )
    
    model.compile(
        optimizer=Adam(learning_rate=hp['learning_rate']),
        loss={
            'direction': lambda y_true, y_pred: asymmetric_focal_loss(
                y_true, y_pred,
                alpha=list(class_weights.values()),
                gamma=hp['gamma']
            ),
            'high': 'huber',
            'low': 'huber'
        },
        loss_weights=[hp['loss_weights_dir'], 0.5*(1-hp['loss_weights_dir']), 0.5*(1-hp['loss_weights_dir'])],
        metrics={'direction': [DirectionAccuracy()]}
    )

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=hp['batch_size'],
        callbacks=[
            TFKerasPruningCallback(trial, 'val_direction_accuracy'),
            EarlyStopping(monitor='val_direction_accuracy', patience=10, mode='max')
        ],
        verbose=0
    )

    return max(history.history['val_direction_accuracy'])

# 6. Основной запуск
if __name__ == "__main__":
    # Загружаем данные
    data = df
    scaler = RobustScaler()
    scaled_data = scaler.fit_transform(data)

    # Балансировка классов
    X, y = create_sequences(scaled_data, window_size=50)
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(np.argmax(y[0], axis=1)),
        y=np.argmax(y[0], axis=1)
    )
    class_weights = dict(enumerate(class_weights))

    # Оптимизация
    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
    study.optimize(objective, n_trials=50, timeout=3600*3)
    
    print(f"Лучшие параметры: {study.best_params}")
    print(f"Лучший результат: {study.best_value:.4f}")


[I 2025-01-26 11:10:47,159] A new study created in memory with name: no-name-3f4a4d24-738e-46e9-83d6-9741bde306df
[W 2025-01-26 11:12:04,728] Trial 0 failed with parameters: {'window_size': 85, 'lstm_units1': 256, 'lstm_units2': 32, 'dropout_rate': 0.24109241705728152, 'learning_rate': 3.425496174487925e-05, 'gamma': 1.1859461869076606, 'batch_size': 128, 'loss_weights_dir': 0.7791428516518702} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\79112\.conda\envs\myenv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\79112\AppData\Local\Temp\ipykernel_12104\2727031208.py", line 129, in objective
    history = model.fit(
              ^^^^^^^^^^
  File "c:\Users\79112\.conda\envs\myenv\Lib\site-packages\keras\src\utils\traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  F

KeyboardInterrupt: 

In [ ]:
def train_final_model(best_params):
    # Загрузка данных
    data = df  # Заменить на реальные данные
    scaler = RobustScaler()
    scaled_data = scaler.fit_transform(data)

    # Создание последовательностей с лучшим window_size
    X, y = create_sequences(scaled_data, best_params['window_size'])
    
    # Разделение данных
    split = int(0.8 * len(X))
    X_train, X_val = X[:split], X[split:]
    y_train = [y[0][:split], y[1][:split], y[2][:split]]
    y_val = [y[0][split:], y[1][split:], y[2][split:]]

    # Создание модели с лучшими параметрами
    model = create_enhanced_model(
        input_shape=(best_params['window_size'], data.shape[1]),
        lstm_units1=best_params['lstm_units1'],
        lstm_units2=best_params['lstm_units2'],
        dropout_rate=best_params['dropout_rate']
    )

    # Компиляция с оптимальными настройками
    model.compile(
        optimizer=Adam(learning_rate=best_params['learning_rate']),
        loss={
            'direction': lambda y_true, y_pred: asymmetric_focal_loss(
                y_true, y_pred,
                alpha=list(class_weights.values()),
                gamma=best_params['gamma']
            ),
            'high': 'huber',
            'low': 'huber'
        },
        loss_weights=[best_params['loss_weights_dir'], 
                     0.5*(1-best_params['loss_weights_dir']), 
                     0.5*(1-best_params['loss_weights_dir'])],
        metrics={'direction': [DirectionAccuracy(), 'accuracy']}
    )

    # Обучение с расширенными callback'ами
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=best_params['batch_size'],
        callbacks=[
            EarlyStopping(
                monitor='val_direction_direction_accuracy',
                patience=15,
                mode='max',
                restore_best_weights=True
            ),
            ReduceLROnPlateau(
                monitor='val_direction_direction_accuracy',
                factor=0.3,
                patience=7,
                mode='max'
            )
        ]
    )
    
    return model, history

# Ваши лучшие параметры
best_params = {
    'window_size': 57,
    'lstm_units1': 64,
    'lstm_units2': 64,
    'dropout_rate': 0.43,
    'learning_rate': 0.000182,
    'gamma': 2.756,
    'batch_size': 64,
    'loss_weights_dir': 0.465
}

# Запуск обучения
final_model, training_history = train_final_model(best_params)

# Визуализация результатов
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(training_history.history['direction_direction_accuracy'], label='Train')
plt.plot(training_history.history['val_direction_direction_accuracy'], label='Validation')
plt.title('Direction Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(training_history.history['loss'], label='Train Loss')
plt.plot(training_history.history['val_loss'], label='Validation Loss')
plt.title('Loss Dynamics')
plt.legend()
plt.show()


Epoch 1/100
434/434 ━━━━━━━━━━━━━━━━━━━━ 36s 65ms/step - direction_accuracy: 0.1775 - direction_direction_accuracy: 0.1775 - direction_loss: -1.6364 - high_loss: 0.2408 - loss: -0.6511 - low_loss: 0.1699 - val_direction_accuracy: 0.1301 - val_direction_direction_accuracy: 0.1301 - val_direction_loss: -31.7305 - val_high_loss: 0.5981 - val_loss: -14.4295 - val_low_loss: 0.6549 - learning_rate: 1.8200e-04
Epoch 2/100
434/434 ━━━━━━━━━━━━━━━━━━━━ 28s 64ms/step - direction_accuracy: 0.1243 - direction_direction_accuracy: 0.1243 - direction_loss: -45.0160 - high_loss: 1.9639 - loss: -20.1411 - low_loss: 0.9948 - val_direction_accuracy: 0.1301 - val_direction_direction_accuracy: 0.1301 - val_direction_loss: -97.4840 - val_high_loss: 0.8520 - val_loss: -44.9292 - val_low_loss: 0.7405 - learning_rate: 1.8200e-04
Epoch 3/100
434/434 ━━━━━━━━━━━━━━━━━━━━ 30s 70ms/step - direction_accuracy: 0.1247 - direction_direction_accuracy: 0.1247 - direction_loss: -110.6614 - high_loss: 0.6091 - loss: -51.2

KeyboardInterrupt: 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, concatenate, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import RobustScaler
from sklearn.utils.class_weight import compute_class_weight

# 1. Кастомная метрика для точности направления
class DirectionAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='direction_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.correct = self.add_weight(name='correct', initializer='zeros')
        self.total = self.add_weight(name='total', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_class = tf.argmax(y_pred, axis=1)
        y_true_class = tf.argmax(y_true, axis=1)
        matches = tf.cast(tf.equal(y_pred_class, y_true_class), tf.float32)
        self.correct.assign_add(tf.reduce_sum(matches))
        self.total.assign_add(tf.cast(tf.size(matches), tf.float32))

    def result(self):
        return self.correct / self.total

    def reset_states(self):
        self.correct.assign(0.0)
        self.total.assign(0.0)

# 2. Улучшенная функция потерь
def asymmetric_focal_loss(y_true, y_pred, alpha=[1.0, 1.0, 1.0], gamma=2.0):
    ce_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    probs = tf.reduce_max(y_true * y_pred, axis=1)
    alpha_weights = tf.reduce_sum(y_true * alpha, axis=1)
    focal_loss = alpha_weights * (1 - probs)**gamma * ce_loss
    return tf.reduce_mean(focal_loss)

# 3. Генератор временных окон с аугментацией
def create_sequences(data, window_size, step=1):
    sequences = []
    targets_dir = []
    targets_high = []
    targets_low = []
    
    for i in range(window_size, len(data)-1):
        sequences.append(data[i-window_size:i])
        targets_dir.append(data[i, 0:3])
        targets_high.append(data[i+1, 3])  # Прогноз на следующий шаг
        targets_low.append(data[i+1, 4])
    
    return (np.array(sequences),
            [np.array(targets_dir),
             np.array(targets_high).reshape(-1,1),
             np.array(targets_low).reshape(-1,1)])

# 4. Улучшенная архитектура модели
def create_trading_model(input_shape):
    inputs = Input(shape=input_shape)
    
    # Ветвь временных зависимостей
    x = LSTM(128, return_sequences=True, dropout=0.3)(inputs)
    x = BatchNormalization()(x)
    x = LSTM(64, dropout=0.2)(x)
    
    # Ветвь технических индикаторов
    tech = inputs[:, -1, -3:]  # Последние 3 технических индикатора
    tech = Dense(16, activation='selu')(tech)
    
    # Объединение ветвей
    merged = concatenate([x, tech])
    merged = Dense(64, activation='selu')(merged)
    merged = Dropout(0.3)(merged)
    
    # Выходы
    direction = Dense(32, activation='selu')(merged)
    direction = Dense(3, activation='softmax', name='direction')(direction)
    
    hl = Dense(32, activation='selu')(merged)
    high = Dense(1, activation='linear', name='high')(hl)
    low = Dense(1, activation='linear', name='low')(hl)
    
    return Model(inputs, [direction, high, low])

# 5. Полный пайплайн
def main():
    # Загрузка и подготовка данных
    data = df
    scaler = RobustScaler()
    scaled_data = scaler.fit_transform(data)
    
    # Создание последовательностей
    window_size = 57
    X, y = create_sequences(scaled_data, window_size)
    
    # Балансировка классов
    classes = np.unique(np.argmax(y[0], axis=1))  # Автоматическое определение классов
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=classes,  # Должен быть numpy array
        y=np.argmax(y[0], axis=1)
    )
    class_weights = dict(zip(classes, class_weights))
    
    # Разделение данных
    split = int(0.8 * len(X))
    X_train, X_val = X[:split], X[split:]
    y_train, y_val = [y[0][:split], y[1][:split], y[2][:split]], \
                     [y[0][split:], y[1][split:], y[2][split:]]
    
    # Создание и компиляция модели
    model = create_trading_model((window_size, 16))
    model.compile(
        optimizer=Adagrad(learning_rate=0.00018206566517784672),
        loss={
            'direction': lambda y_true, y_pred: asymmetric_focal_loss(
                y_true, y_pred, 
                alpha=list(class_weights.values())  # Передаем веса как параметр
            ),
            'high': 'huber',
            'low': 'huber'
        },
        loss_weights=[0.6, 0.2, 0.2],
        metrics={
            'direction': [DirectionAccuracy(), 'accuracy'],
            'high': ['mae'],
            'low': ['mae']
        }
    )

    # В вызове fit() удаляем class_weight
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=64,
        callbacks=[
            EarlyStopping(
                monitor='val_direction_accuracy',
                patience=12,
                mode='max',
                restore_best_weights=True
            ),
            ReduceLROnPlateau(
                monitor='val_direction_accuracy',
                factor=0.5,
                patience=5,
                mode='max'
            )
        ]
    )
    

    
    # Визуализация
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(history.history['direction_accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_direction_accuracy'], label='Val Accuracy')
    plt.axhline(0.5, color='red', linestyle='--', label='50% Threshold')
    plt.title('Direction Accuracy')
    plt.legend()
    
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Total Loss')
    plt.legend()
    plt.show()

if __name__ == "__main__":
    main()


Epoch 1/50


ValueError: Input 0 of layer "functional_3" is incompatible with the layer: expected shape=(None, 57, 16), found shape=(None, 57, 14)

In [ ]:
def scale_data(dummy_data):
    """Масштабирование данных."""
    scaler = RobustScaler()
    return scaler.fit_transform(dummy_data)

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    """Динамический график изменения learning rate с warm-up."""
    def __init__(self, initial_lr=1e-4, warmup_steps=1000):
        super().__init__()
        self.initial_lr = initial_lr
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        return tf.minimum(self.initial_lr * tf.sqrt(step / self.warmup_steps), self.initial_lr)

def weighted_asymmetric_loss(y_true, y_pred):
    """Асимметричная взвешенная cross-entropy loss."""
    true_idx = tf.argmax(y_true, axis=1)
    weights = tf.where(true_idx == 2, 1.5, tf.where(true_idx == 1, 0.7, 1.0))
    ce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    return tf.reduce_mean(ce * weights)

def mare_loss(y_true, y_pred):
    """Mean Absolute Relative Error (MARE)."""
    return tf.reduce_mean(tf.abs((y_true - y_pred) / (y_true + 1e-8)))

class CustomLoss(tf.keras.losses.Loss):
    """Комбинированная функция потерь."""
    def __init__(self, alpha=0.3, beta=0.2, name="custom_loss"):
        super().__init__(name=name)
        self.alpha = alpha
        self.beta = beta

    def call(self, y_true, y_pred):
        direction_true, high_true, low_true = y_true
        direction_pred, high_pred, low_pred = y_pred

        ce_loss = weighted_asymmetric_loss(direction_true, direction_pred)
        hl_loss = 0.5 * (mare_loss(high_true, high_pred) + mare_loss(low_true, low_pred))
        return ce_loss + self.alpha * hl_loss

def prepare_data(data, window_size=30):
    X, y_dir, y_high, y_low = [], [], [], []
    for i in range(window_size, len(data)):
        X.append(data[i - window_size : i])
        y_dir.append(data[i, 0:3])
        y_high.append(data[i, 3])  # High
        y_low.append(data[i, 4])   # Low

    # Возвращаем данные ПОСЛЕ завершения цикла
    X = np.array(X)
    y_dir = np.array(y_dir)
    y_high = np.array(y_high).reshape(-1, 1)
    y_low = np.array(y_low).reshape(-1, 1)
    
    return X, [y_dir, y_high, y_low]

def create_lstm_model(input_shape, n_units=64, freeze_encoder=False):
    """Создание мультивыходной LSTM-модели."""
    inputs = Input(shape=input_shape, name="input_layer")
    x = LSTM(n_units, return_sequences=True, name="lstm")(inputs)

    # Направление
    direction_branch = LSTM(n_units // 2, return_sequences=False, name="lstm2")(x)
    direction_out = Dense(3, activation="softmax", name="direction")(direction_branch)

    # Прогноз High/Low
    atr = Lambda(lambda x: x[:, :, -1:], name="atr_slice")(inputs)  # Последний столбец — ATR
    atr = Reshape((input_shape[0], 1), name="atr_reshape")(atr)     # Приводим к совместимой форме
    hl_concat = concatenate([x, atr], axis=-1, name="concat_atr")

    hl_hidden = Dense(32, activation="selu", name="hl_dense")(hl_concat)
    high_out = Dense(1, activation="linear", name="high")(hl_hidden)
    low_out = Dense(1, activation="linear", name="low")(hl_hidden)

    model = Model(inputs=inputs, outputs=[direction_out, high_out, low_out])
    if freeze_encoder:
        for layer in model.layers[:-6]:
            layer.trainable = False

    return model

# -------------------------------------------------------------------------
# 2. Подготовка данных
# -------------------------------------------------------------------------

# Сгенерируем данные для примера
np.random.seed(42)  # Для воспроизводимости
dummy_data = np.random.rand(1000, 16)  # Пример ваших данных (1000 наблюдений, 16 признаков)

# Масштабирование
scaled_data = scale_data(dummy_data)

# Создание временных окон
window_size = 30
X_seq, y_seq = prepare_data(scaled_data, window_size=window_size)

# Разделение данных
total_samples = X_seq.shape[0]
train_end = int(0.7 * total_samples)
val_end = int(0.85 * total_samples)

X_train = X_seq[:train_end]
X_val = X_seq[train_end:val_end]
X_test = X_seq[val_end:]

y_dir_train, y_high_train, y_low_train = y_seq[0][:train_end], y_seq[1][:train_end], y_seq[2][:train_end]
y_dir_val,   y_high_val,   y_low_val   = y_seq[0][train_end:val_end], y_seq[1][train_end:val_end], y_seq[2][train_end:]
y_dir_test,  y_high_test,  y_low_test  = y_seq[0][val_end:], y_seq[1][val_end:], y_seq[2][val_end:]

# -------------------------------------------------------------------------
# 3. Создание и обучение модели
# -------------------------------------------------------------------------

# Создание модели
model = create_lstm_model(input_shape=(window_size, 16), n_units=64)

# Компиляция модели
lr_schedule = CustomSchedule()
optimizer = Adam(learning_rate=lr_schedule)

model.compile(
    optimizer=optimizer,
    loss={
        "direction": weighted_asymmetric_loss,
        "high": "mae",
        "low": "mae"
    },
    loss_weights=[0.7, 0.15, 0.15],
    metrics={
        "direction": "accuracy",
        "high": "mae",
        "low": "mae"
    }
)

# Обучение модели
train_targets = {"direction": y_dir_train, "high": y_high_train, "low": y_low_train}
val_targets = {"direction": y_dir_val, "high": y_high_val, "low": y_low_val}

callbacks = [
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1),
    EarlyStopping(monitor="val_direction_accuracy", patience=10, verbose=1, restore_best_weights=True)
]

history_stage1 = model.fit(
    X_train,
    train_targets,
    epochs=10,
    batch_size=64,
    validation_data=(X_val, val_targets),
    callbacks=callbacks,
    verbose=1
)

# -------------------------------------------------------------------------
# 4. Оценка модели
# -------------------------------------------------------------------------

print("\nModel Evaluation:")
test_results = model.evaluate(X_test, {"direction": y_dir_test, "high": y_high_test, "low": y_low_test})
print("Test Results:", test_results)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - direction_accuracy: 0.3459 - direction_loss: 0.0056 - high_loss: 0.5104 - high_mae: 0.5105 - loss: 0.1604 - low_loss: 0.5323 - low_mae: 0.5325  

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 145
'y' sizes: 145, 145, 291
